<a href="https://colab.research.google.com/github/kangwonlee/2018pycpp/blob//50.under-the-hood/28.working_with_bits_struct_union.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Revisiting `struct` and `union`



* `struct` is a group of data.  An array is also a group of data.  Difference is, to access data in `struct`, we would use *field name*s; for arrays, indices.



* Also, *field*s of struct may have different types; for arrays, all same types.



* Fields of `unions` share the same memory.



#### `union`: Little endian vs Big endian



* Let's see how our computer stores a multi-byte integer.



* Using `union` we can make a multi-byte integer and an array of characters share the memory.



In [ ]:
%%writefile little_or_big.cpp
// The directive above would write following text into a file

#include <cstdio>
#include <cstdint>

// i and c[4] would share the same memory
union little_big_tag{
    uint32_t i;
    char c[4];
};


int32_t main(const int32_t argn, const char * argv[]){
    
    // an instance of the union
    union little_big_tag s;

    // See if both int and char[] member share the same memory
    printf("&(s.i) = %08lx\n", (uint64_t) (&(s.i)));
    printf("&(s.c) = %08lx\n", (uint64_t) (&(s.c)));
    
    printf(
        "They share the same memory if non-zero: %x\n", 
        (
            (uint64_t) (&(s.i)) == (uint64_t) (&(s.c))
        )
    );

    // assign an integer to the integer member of s
    s.i = 0x01020304;
    // reading the memory as one 32 bit integer
    printf("s.i = %08x\n", s.i);
    // as four 8 bit characters
    printf("s.c = %02x %02x %02x %02x\n", s.c[0], s.c[1], s.c[2], s.c[3]);

    // check address of each member of char[4]
    // which part of integer is in which part of array?
    int32_t i = 0;
    for (i = 0; 4 > i; i++){
        printf("&(s.c[%d]) = %08lx\n", i, (uint64_t) (&(s.c[i])));
    }
}



In [ ]:
!g++ -Wall -g -std=c++14 little_or_big.cpp -o little_or_big -Wa,-adhln=little_or_big.s



In [ ]:
!./little_or_big



#### 16bit color using `struct` and `union`



* For each field of struct, we may specify its length in **bits**.



* Followinging could be one example for the 16bit color information.



``` C++
#include <cstdint>

// Josh Kunz, Bit-field Packing in GCC and Clang
//     https://jkz.wtf/bit-field-packing-in-gcc-and-clang

struct high_color_tag{
    uint16_t red:5;
    uint16_t green:6;
    uint16_t blue:5;
};
```



* We can see that the sum of bits is `5 + 6 + 5 = 16`; two bytes.



* Following C++ code would show some examples.



In [ ]:
%%writefile bit_field_struct.cpp
#include <bitset>
#include <cstdint>
#include <cstdlib>
#include <iomanip>
#include <iostream>

// Josh Kunz, Bit-field Packing in GCC and Clang, 
//     https://jkz.wtf/bit-field-packing-in-gcc-and-clang

// Bit field struct
struct high_color_tag{
    uint16_t red:5;
    uint16_t green:6;
    uint16_t blue:5;
};

// Let the struct share memory with an unsigned 16bit integer
union high_color_union_tag{
    struct high_color_tag rgb_struct;
    uint16_t hex;
};


int32_t main(const int32_t argn, const char * argv[]){
    // realize the union
    union high_color_union_tag rgb_union;
    // number of examples
    const int32_t n = 10;

    // variables for r g b colors and index
    uint32_t r = 0, g = 0, b = 0, i = 0;
    
    // table header
    std::cout << "| `r` | `g` | `b` | `hex` |" << '\n';
    std::cout << "|:---:|:---:|:---:|:-----:|" << '\n';

    // example loop
    for(i = 0; n > i; ++i){
        // generate color values
        // 1 << 5 == ?
        r = rand() % (1 << 5);
        g = rand() % (1 << 6);
        b = rand() % (1 << 5);

        // set values to the struct fields
        rgb_union.rgb_struct.red = r;
        rgb_union.rgb_struct.green = g;
        rgb_union.rgb_struct.blue = b;

        // bit patterns of color values
        // https://stackoverflow.com/questions/7349689
        std::bitset<5> b_r(r);
        std::bitset<6> b_g(g);
        std::bitset<6> b_b(b);

        // bit pattern of the 16bit integer
        std::bitset<16> b_hex(rgb_union.hex);

        // print this example as a row
        std::cout << "| " << std::hex << std::setw(6) << b_r
                    << " | " << std::setw(6) << b_g
                    << " | " << std::setw(6) << b_b
                    << " | " << std::setw(16) << b_hex
                    << " |\n";
    }

    // std::cout << "sizeof(rgb_union.rgb_struct)" << sizeof(rgb_union.rgb_struct) << '\n';

}



In [ ]:
!g++ -Wall -g -std=c++14 bit_field_struct.cpp -o bit_field_struct -Wa,-adhln=bit_field_struct.s



In [ ]:
%%bash
# The directive above would run following as bash commands

# Detect OS type because OSX may need different options
# https://stackoverflow.com/questions/3466166/how-to-check-if-running-in-cygwin-mac-or-linux/18790824

# obtain system information
unameOut="$(uname -s)"

# detect system type
case "${unameOut}" in
    Linux*)     machine=Linux;;
    Darwin*)    machine=Mac;;
    CYGWIN*)    machine=Cygwin;;
    MINGW*)     machine=MinGw;;
    *)          machine="UNKNOWN:${unameOut}"
esac

if [ $machine == "Linux" ]; then
    # build command for Linux
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct -Wa,-adhln=bit_field_struct.s
elif [ "Mac" == $machine ]; then
    # build command for OSX
    # https://stackoverflow.com/questions/10990018/
    clang++ -S -mllvm --x86-asm-syntax=intel bit_field_struct.cpp
    clang++ -Wall -g bit_field_struct.cpp -o bit_field_struct
else
    # Otherwise
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct.s -S
    g++ -Wall -g bit_field_struct.cpp -o ./bit_field_struct
fi



In [ ]:
%%bash --out rgb_16 --err error
./bit_field_struct



In [ ]:
assert (("error" not in dir()) or (not error)), error



In [ ]:
import IPython.display as disp
disp.display(disp.Markdown(rgb_16))



* Now let's take a look at some lines of the assembly code.



```
  44:bit_field_struct.cpp ****
  45:bit_field_struct.cpp ****   // set values to the struct fields
  46:bit_field_struct.cpp ****   rgb_union.rgb_struct.red = r;
 568                            .loc 6 46 0
 569 00e4 8B45BC                movl    -68(%rbp), %eax
 570 00e7 83E01F                andl    $31, %eax
 571 00ea 83E01F                andl    $31, %eax
 572 00ed 89C2                  movl    %eax, %edx
 573 00ef 0FB645B2              movzbl  -78(%rbp), %eax
 574 00f3 83E0E0                andl    $-32, %eax
 575 00f6 09D0                  orl     %edx, %eax
 576 00f8 8845B2                movb    %al, -78(%rbp)
  47:bit_field_struct.cpp ****   rgb_union.rgb_struct.green = g;
 577                            .loc 6 47 0
 578 00fb 8B45C0                movl    -64(%rbp), %eax
 579 00fe 83E03F                andl    $63, %eax
 580 0101 0FB6C0                movzbl  %al, %eax
 581 0104 83E03F                andl    $63, %eax
 582 0107 C1E005                sall    $5, %eax
 583 010a 89C2                  movl    %eax, %edx
 584 010c 0FB745B2              movzwl  -78(%rbp), %eax
 585 0110 66251FF8              andw    $-2017, %ax
 586 0114 09D0                  orl     %edx, %eax
 587 0116 668945B2              movw    %ax, -78(%rbp)
 48:bit_field_struct.cpp ****    rgb_union.rgb_struct.blue = b;
 588                            .loc 6 48 0
 589 011a 8B45C4                movl    -60(%rbp), %eax
 590 011d 83E01F                andl    $31, %eax
 591 0120 8D14C500000000        leal    0(,%rax,8), %edx
 592 0127 0FB645B3              movzbl  -77(%rbp), %eax
 593 012b 83E007                andl    $7, %eax
 594 012e 09D0                  orl     %edx, %eax
 595 0130 8845B3                movb    %al, -77(%rbp) 
```



* Please compare with the previous example.



```
  25:bitwise_16bit_color.cpp ****
  26:bitwise_16bit_color.cpp **** // set red value from bit 0 ~ 4
  27:bitwise_16bit_color.cpp ****         hex |= (r & 0x1F);
 567                            .loc 6 27 0
 568 00d6 8B45B8                movl    -72(%rbp), %eax
 569 00d9 83E01F                andl    $31, %eax
 570 00dc 0945C4                orl     %eax, -60(%rbp)
  28:bitwise_16bit_color.cpp ****
  29:bitwise_16bit_color.cpp **** // set green value from bit 5 ~ 10
  30:bitwise_16bit_color.cpp ****         hex |= ((g & 0x3F) << 5);
 571                            .loc 6 30 0
 572 00df 8B45BC                movl    -68(%rbp), %eax
 573 00e2 C1E005                sall    $5, %eax
 574 00e5 25E00700              andl    $2016, %eax
 574      00
 575 00ea 0945C4                orl     %eax, -60(%rbp)
  31:bitwise_16bit_color.cpp ****
  32:bitwise_16bit_color.cpp **** // set blue value from bit 11 ~ 15
  33:bitwise_16bit_color.cpp ****         hex |= ((b & 0x1F) << 11);
 576                            .loc 6 33 0
 577 00ed 8B45C0                movl    -64(%rbp), %eax
 578 00f0 C1E00B                sall    $11, %eax
 579 00f3 0FB7C0                movzwl  %ax, %eax
 580 00f6 0945C4                orl     %eax, -60(%rbp)
```

